<a href="https://colab.research.google.com/github/BharatXBhalla/NLP-CDAC/blob/main/Preprocessing/Cleaning_Textual_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
#read the dataset file
df = pd.read_csv('/content/Reviews.csv', encoding='utf-8', sep=',') 
#drop the duplicate and na values from the records
print(df.shape)
df.drop_duplicates(subset=['Text'],inplace=True)
print(df.shape)
df.dropna(axis=0,inplace=True)
print(df.shape)

(11162, 10)
(10610, 10)
(10610, 10)


In [ ]:
input_data = df.loc[:,'Text']
target_data = df.loc[:,'Summary']
target_data.replace('', np.nan, inplace=True)

In [ ]:
input_data.head()

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: Text, dtype: object

In [ ]:
target_data.head()

0    Good Quality Dog Food
1        Not as Advertised
2    "Delight" says it all
3           Cough Medicine
4              Great taffy
Name: Summary, dtype: object

In [ ]:
print(input_data[0])
target_data[0]

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


'Good Quality Dog Food'

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.stem import LancasterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import pickle

contractions= pickle.load(open("/content/contractions.pkl","rb"))['contractions']
#initialize stop words and LancasterStemmer
stop_words=set(stopwords.words('english'))
stemm=LancasterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from bs4 import BeautifulSoup

BeautifulSoup("hello <h>hhe</h>", "lxml").text

'hello hhe'

In [ ]:
word_tokenize("hellow rold!")

['hellow', 'rold', '!']

In [ ]:
list(filter(lambda w:(w.isalpha() and len(w)>=3),['hellow', 'rold', '!!!!']))

['hellow', 'rold']

In [ ]:
[stemm.stem("Hwllo"),stemm.stem("Helling"),stemm.stem("Welcome"),stemm.stem("Increase")]

['hwllo', 'hel', 'welcom', 'increas']

In [ ]:
def clean(texts,src):

  texts = BeautifulSoup(texts, "lxml").text

  words=word_tokenize(texts.lower())

  words= list(filter(lambda w:(w.isalpha() and len(w)>=3),words))

  words= [contractions[w] if w in contractions else w for w in words ]

  if src=="inputs":
    words= [stemm.stem(w) for w in words if w not in stop_words]
  else:
    words= [w for w in words if w not in stop_words]
  return words

In [ ]:
input_texts=[]
target_texts=[]
input_words=[]
target_words=[]

for in_txt,tr_txt in zip(input_data,target_data):
  in_words= clean(in_txt,"inputs")
  input_texts+= [' '.join(in_words)]
  input_words+= in_words
  #add 'sos' at start and 'eos' at end of text
  tr_words= clean("sos "+tr_txt+" eos","target")
  target_texts+= [' '.join(tr_words)]
  target_words+= tr_words


In [ ]:
#store only unique words from input and target list of words
input_words = sorted(list(set(input_words)))
target_words = sorted(list(set(target_words)))
num_in_words = len(input_words) #total number of input words
num_tr_words = len(target_words) #total number of target words

In [ ]:
from statistics import mode

#get the length of the input and target texts which appears most often  
max_in_len = mode([len(i) for i in input_texts])
max_tr_len = mode([len(i) for i in target_texts])

In [ ]:
max_in_len

73

In [ ]:
max_tr_len

17

In [ ]:
print("number of input words : ",num_in_words)
print("number of target words : ",num_tr_words)
print("maximum input length : ",max_in_len)
print("maximum target length : ",max_tr_len)

number of input words :  10886
number of target words :  4414
maximum input length :  73
maximum target length :  17
